In [1]:
import random
import numpy as np
import tensorflow as tf
import os

# Set seeds
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [ ]:
import pandas as pd
import numpy as np
import os

meta = pd.read_csv('Train_Test_Split.csv')
meta = meta[meta['label'].isin(['AD', 'Healthy'])]

train_meta = meta[meta['split'] == 'train']
test_meta = meta[meta['split'] == 'test']

def load_and_segment(subject_id, data_dir='Data_sampled_128HZ', segment_len=1024):
    file_path = os.path.join(data_dir, f"{subject_id}_data.npy")
    data = np.load(file_path)
    _, time_steps = data.shape
    num_segments = time_steps // segment_len
    if num_segments == 0:
        return np.empty((0, 19, segment_len))
    data = data[:, :num_segments * segment_len]
    segments = data.reshape(19, num_segments, segment_len).transpose(1, 0, 2)
    return segments

def process_data(meta_df, data_dir='Data_sampled_128HZ'):
    X = []
    y = []
    label_map = {'AD': 1, 'Healthy': 0}
    for _, row in meta_df.iterrows():
        segments = load_and_segment(row['subject_id'], data_dir)
        if segments.shape[0] == 0:
            continue
        X.append(segments)
        label = label_map[row['label']]
        one_hot = np.eye(2)[label]
        y.extend([one_hot] * segments.shape[0])
    X = np.concatenate(X, axis=0)
    y = np.array(y)
    return X, y
X_train, y_train = process_data(train_meta)
X_test, y_test = process_data(test_meta)
X_train = (X_train * 1e6) - np.mean(X_train * 1e6, axis=2, keepdims=True)
X_test = (X_test * 1e6) - np.mean(X_test * 1e6, axis=2, keepdims=True)

In [ ]:
from tensorflow.keras.models import load_model, Model

def load_and_trim_model(path):
    model = load_model(path)
    model.pop() 
    model.trainable = False  # freeze
    return model
bilstm_model = load_and_trim_model('Models\Final_Bilstm_model.keras')
cnn_time_model = load_and_trim_model('Models\Final_CNNSpatial_model.keras')
cnn_freq_model = load_and_trim_model('Models\Final_CNNSpectral_model.keras')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, GlobalAveragePooling1D, Concatenate, Lambda, Embedding
from tensorflow.keras.models import Model

def build_fusion_model_with_transformer(bilstm_model, cnn_time_model, cnn_freq_model, common_dim=128, num_heads=4, ff_dim=256, dropout_rate=0.3):
    bilstm_input = Input(shape=bilstm_model.output_shape[1:])
    cnn_time_input = Input(shape=cnn_time_model.output_shape[1:])
    cnn_freq_input = Input(shape=cnn_freq_model.output_shape[1:])

    bilstm_proj = Dense(common_dim)(bilstm_input)
    cnn_time_proj = Dense(common_dim)(cnn_time_input)
    cnn_freq_proj = Dense(common_dim)(cnn_freq_input)

    x = Lambda(lambda t: tf.stack(t, axis=1))([bilstm_proj, cnn_time_proj, cnn_freq_proj])

    for _ in range(2):
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=common_dim)(x, x)
        x = LayerNormalization(epsilon=1e-6)(x + attn_output)

        ffn_output = Dense(ff_dim, activation='relu')(x)
        ffn_output = Dense(common_dim)(ffn_output)
        x = LayerNormalization(epsilon=1e-6)(x + ffn_output)

    x = GlobalAveragePooling1D()(x)
    x = Dropout(dropout_rate)(x)
    out = Dense(2, activation='softmax')(x)

    model = Model(inputs=[bilstm_input, cnn_time_input, cnn_freq_input], outputs=out)
    return model


In [ ]:
fusion_model = build_fusion_model_with_transformer(
    bilstm_model, cnn_time_model, cnn_freq_model,
    common_dim=128, num_heads=4, ff_dim=256, dropout_rate=0.3
)

from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=1e-3) 

fusion_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy','AUC'])


In [ ]:
#BiLSTM Inputs
top_channels = [14, 2, 0, 18, 4]
X_train_selected = X_train[:, top_channels, :].transpose(0, 2, 1)  # (samples, 1024, 5)
X_test_selected = X_test[:, top_channels, :].transpose(0, 2, 1)    # (samples, 1024, 5)
X_bilstm_input = X_train_selected
# CNNSPatial Inputs
X_train_cnn = X_train[..., np.newaxis]  # shape: (N, 19, 1024, 1)
X_test_cnn = X_test[..., np.newaxis]
X_cnn_time_input = X_train_cnn
#CNNSpectral Inputs
from scipy.signal import welch

def compute_spectral_features(X, fs=128, nperseg=256):
    num_segments, num_channels, num_samples = X.shape
    psd_all = []

    for seg in X:
        seg_psd = []
        for ch in seg:
            freqs, psd = welch(ch, fs=fs, nperseg=nperseg)
            seg_psd.append(psd)
        psd_all.append(seg_psd)

    psd_all = np.array(psd_all)  
    psd_all = np.log1p(psd_all)  
    return psd_all, freqs

X_train_spec, freqs = compute_spectral_features(X_train)
X_test_spec, _ = compute_spectral_features(X_test)

X_train_spec = X_train_spec[..., np.newaxis]  # shape: (N, 19, freq_bins, 1)
X_test_spec = X_test_spec[..., np.newaxis]

X_cnn_freq_input = X_train_spec

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
callbacks = [
    EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4)
]

In [ ]:
X_bilstm_feat = bilstm_model.predict(X_bilstm_input)
X_cnn_time_feat = cnn_time_model.predict(X_cnn_time_input)
X_cnn_freq_feat = cnn_freq_model.predict(X_cnn_freq_input)

164/164 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [ ]:
fusion_model.fit(
    [X_bilstm_feat, X_cnn_time_feat, X_cnn_freq_feat],
    y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks = callbacks
)


Epoch 1/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 17s 51ms/step - AUC: 0.9775 - accuracy: 0.9468 - loss: 0.1702 - val_AUC: 0.9778 - val_accuracy: 0.9362 - val_loss: 0.2023 - learning_rate: 0.0010
Epoch 2/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - AUC: 0.9982 - accuracy: 0.9888 - loss: 0.0362 - val_AUC: 0.9607 - val_accuracy: 0.9038 - val_loss: 0.2718 - learning_rate: 0.0010
Epoch 3/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - AUC: 0.9967 - accuracy: 0.9901 - loss: 0.0358 - val_AUC: 0.9810 - val_accuracy: 0.9343 - val_loss: 0.2036 - learning_rate: 0.0010
Epoch 4/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - AUC: 0.9975 - accuracy: 0.9896 - loss: 0.0376 - val_AUC: 0.9809 - val_accuracy: 0.9448 - val_loss: 0.1792 - learning_rate: 0.0010
Epoch 5/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - AUC: 0.9989 - accuracy: 0.9937 - loss: 0.0242 - val_AUC: 0.9526 - val_accuracy: 0.8857 - val_loss: 0.3334 - learning_rate: 0.0010
Epoch 6/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - AUC: 0.9984 - accur

In [11]:
X_bilstm_test_feat = bilstm_model.predict(X_test_selected)
X_cnn_time_test_feat = cnn_time_model.predict(X_test_cnn)
X_cnn_freq_test_feat = cnn_freq_model.predict(X_test_spec)

37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 105ms/step
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [12]:
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score

y_pred = fusion_model.predict([X_bilstm_test_feat, X_cnn_time_test_feat, X_cnn_freq_test_feat])
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print("ROC AUC:", roc_auc_score(y_true, y_pred[:, 1]))
print("Average Precision:", average_precision_score(y_true, y_pred[:, 1]))
print(classification_report(y_true, y_pred_classes))

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
ROC AUC: 0.9311796069973639
Average Precision: 0.9367258349276433
              precision    recall  f1-score   support

           0       0.89      0.76      0.82       535
           1       0.82      0.92      0.87       624

    accuracy                           0.85      1159
   macro avg       0.86      0.84      0.85      1159
weighted avg       0.85      0.85      0.85      1159



In [ ]:
output_folder = "Models"
os.makedirs(output_folder,exist_ok=True)
model_path = os.path.join(output_folder,"Final_End_to_End_model.keras")
model.save(model_path)

In [ ]:
from scipy.stats import mode

def patient_level_ensemble_2(fusion_model, bilstm_model, cnn_time_model, cnn_freq_model, meta_df, voting='soft'):
    y_true = []
    y_pred = []
    y_prob = [] 

    for _, row in meta_df.iterrows():
        subject_id = row['subject_id']
        label_str = row['label']
        true_label = 1 if label_str == 'AD' else 0

        segments = load_and_segment(subject_id)
        if segments.shape[0] == 0:
            continue
        segments = (segments * 1e6) - np.mean(segments * 1e6, axis=2, keepdims=True)
        top_channels = [14, 2, 0, 18, 4]
        bilstm_input = segments[:, top_channels, :].transpose(0, 2, 1)
        cnn_time_input = segments[..., np.newaxis]
        spec_feats, _ = compute_spectral_features(segments)
        cnn_freq_input = spec_feats[..., np.newaxis]

        bilstm_feat = bilstm_model.predict(bilstm_input, verbose=0)
        cnn_time_feat = cnn_time_model.predict(cnn_time_input, verbose=0)
        cnn_freq_feat = cnn_freq_model.predict(cnn_freq_input, verbose=0)

        preds = fusion_model.predict([bilstm_feat, cnn_time_feat, cnn_freq_feat], verbose=0)

        if voting == 'soft':
            avg_prob = np.mean(preds, axis=0)
            y_pred.append(np.argmax(avg_prob))
            y_prob.append(avg_prob[1])
        elif voting == 'hard':
            pred_classes = np.argmax(preds, axis=1)
            voted_class = mode(pred_classes, keepdims=True).mode[0]
            y_pred.append(voted_class)
            y_prob.append(np.mean(preds[:, 1]))

        y_true.append(true_label)

    return np.array(y_true), np.array(y_pred), np.array(y_prob)


In [ ]:
yt_hard, yp_hard, prob_hard = patient_level_ensemble_2(fusion_model, bilstm_model, cnn_time_model, cnn_freq_model, test_meta, voting='hard')

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score

def evaluate_predictions(y_true, y_pred, y_prob, voting_type="Soft"):
    print(f"\n=== {voting_type} Voting Results ===")
    print(classification_report(y_true, y_pred))

evaluate_predictions(yt_hard, yp_hard, prob_hard, voting_type="Hard")


=== Hard Voting Results ===
              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.86      1.00      0.92         6

    accuracy                           0.91        11
   macro avg       0.93      0.90      0.91        11
weighted avg       0.92      0.91      0.91        11

